In [19]:
import os
from pyspark.sql import SparkSession

# Set the SPARK_HOME environment variable
os.environ["SPARK_HOME"] = "/usr/local/lib/python3.10/dist-packages/pyspark"

# Set the PYSPARK_SUBMIT_ARGS to include the path to spark-submit and Py4J JAR
os.environ["PYSPARK_SUBMIT_ARGS"] = "--master local[*] --driver-memory 4g --executor-memory 4g --jars /usr/local/lib/python3.10/dist-packages/pyspark/py4j-0.10.9.7.jar pyspark-shell"

# Create a SparkSession
spark = SparkSession.builder \
    .appName("SparkSQL") \
    .getOrCreate()


In [20]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [23]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), header=True, inferSchema=True)


In [24]:
# 2. Create a temporary view of the DataFrame.
df.createOrReplaceTempView("home_sales")


In [26]:
# 3. What is the average price for a four bedroom house sold per year, rounded to two decimal places?
query_3 = """
SELECT YEAR(date), ROUND(AVG(price), 2) AS avg_price
FROM home_sales
WHERE bedrooms = 4
GROUP BY YEAR(date)
ORDER BY YEAR(date)
"""
result_3 = spark.sql(query_3)
result_3.show()

+----------+---------+
|year(date)|avg_price|
+----------+---------+
|      2019| 300263.7|
|      2020|298353.78|
|      2021|301819.44|
|      2022|296363.88|
+----------+---------+



In [28]:
# 4. What is the average price of a home for each year the home was built,
# that have 3 bedrooms and 3 bathrooms, rounded to two decimal places?
query_4 = """
SELECT YEAR(FROM_UNIXTIME(date_built)), ROUND(AVG(price), 2) AS avg_price
FROM home_sales
WHERE bedrooms = 3 AND bathrooms = 3
GROUP BY YEAR(FROM_UNIXTIME(date_built))
ORDER BY YEAR(FROM_UNIXTIME(date_built))
"""

# Run the query
result_4 = spark.sql(query_4)
result_4.show()

+----------------------------------------------------+---------+
|year(from_unixtime(date_built, yyyy-MM-dd HH:mm:ss))|avg_price|
+----------------------------------------------------+---------+
|                                                1970|292003.61|
+----------------------------------------------------+---------+



In [30]:
# 5. What is the average price of a home for each year the home was built,
# that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet, rounded to two decimal places?
query_5 = """
SELECT YEAR(FROM_UNIXTIME(date_built)) AS year_built, ROUND(AVG(price), 2) AS avg_price
FROM home_sales
WHERE bedrooms = 3 AND bathrooms = 3 AND floors = 2 AND sqft_living >= 2000
GROUP BY YEAR(FROM_UNIXTIME(date_built))
ORDER BY YEAR(FROM_UNIXTIME(date_built))
"""

# Run the query
result_5 = spark.sql(query_5)
result_5.show()

+----------+---------+
|year_built|avg_price|
+----------+---------+
|      1970|292538.42|
+----------+---------+



In [31]:
# 6. What is the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000? Order by descending view rating.
# Although this is a small dataset, determine the run time for this query.

start_time = time.time()
query_6 = """
SELECT view, ROUND(AVG(price), 2) AS avg_price
FROM home_sales
WHERE price >= 350000
GROUP BY view
ORDER BY view DESC
"""

# Run the query
result_6 = spark.sql(query_6)
result_6.show()


print("--- %s seconds ---" % (time.time() - start_time))

+----+----------+
|view| avg_price|
+----+----------+
| 100| 1026669.5|
|  99|1061201.42|
|  98|1053739.33|
|  97|1129040.15|
|  96|1017815.92|
|  95| 1054325.6|
|  94| 1033536.2|
|  93|1026006.06|
|  92| 970402.55|
|  91|1137372.73|
|  90|1062654.16|
|  89|1107839.15|
|  88|1031719.35|
|  87| 1072285.2|
|  86|1070444.25|
|  85|1056336.74|
|  84|1117233.13|
|  83|1033965.93|
|  82| 1063498.0|
|  81|1053472.79|
+----+----------+
only showing top 20 rows

--- 0.9060003757476807 seconds ---


In [32]:
# 7. Cache the the temporary table home_sales.
spark.catalog.cacheTable("home_sales")

In [33]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [34]:
# 9. Using the cached data, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the uncached runtime.

start_time = time.time()

query_9 = """
SELECT view, ROUND(AVG(price), 2) AS avg_price
FROM home_sales
GROUP BY view
HAVING AVG(price) >= 350000
ORDER BY view DESC
"""

# Run the query using cached data
start_time_cached = time.time()
result_9_cached = spark.sql(query_9)
runtime_cached = time.time() - start_time_cached

print("--- %s seconds ---" % (time.time() - start_time))


--- 0.07571792602539062 seconds ---


In [39]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
parquet_path = "/content"

In [41]:
# 11. Read the formatted parquet data.
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType

# Define the schema of your Parquet data
parquet_schema = StructType([
    StructField("id", IntegerType(), True),
    StructField("date", StringType(), True),
    # Add other fields here based on your data
])

# Read the Parquet data with the specified schema
parquet_df = spark.read.schema(parquet_schema).parquet(parquet_path)

In [42]:
# 12. Create a temporary table for the parquet data.
parquet_df.createOrReplaceTempView("parquet_home_sales")

In [43]:
# 13. Using the parquet DataFrame, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the cached runtime.

start_time = time.time()

# Run the query using parquet data
start_time_parquet = time.time()
result_9_parquet = spark.sql(query_9)
runtime_parquet = time.time() - start_time_parquet

print("--- %s seconds ---" % (time.time() - start_time))

--- 0.035538434982299805 seconds ---


In [44]:
# 14. Uncache the home_sales temporary table.
spark.catalog.uncacheTable("home_sales")

In [46]:
# 15. Check if the home_sales is no longer cached
print("Are home_sales cached? ", spark.catalog.isCached('home_sales'))

Are home_sales cached?  False
